In [2]:
import os
import pybullet_envs
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
# import panda_gym # Not in requirements, required for panda envs, get latest from here: https://github.com/qgallouedec/panda-gym , don't use version from pypy 

from seagul.zoo3_utils import load_zoo_agent, ALGOS

path_to_zoo = "/home/ty/Documents/GitHub/rl-baselines3-zoo/" # Very hacky but this is what we do for now. use: git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo

#model_dir = os.path.abspath('') + "/keep_agents/bullet_works_rewards/" # Bullet envs with mdim and cdim rewards
#model_dir = os.path.abspath('') + "/keep_agents/default_post_01_001/" # Bullet with no extra reward, just policy refinement with ARS  
model_dir = "/home/ty/Documents/GitHub/policy_refinement/keep_agents/train_all/"

In [3]:
# Iterate through save directory and load everything

model_dict = {}
env_dict = {}

for algo_file in os.scandir(model_dir):
    algo_dir = algo_file.path
    algo = algo_dir.split("/")[-1]
    model_dict[algo] = {}
    env_dict[algo] = {}
    for env_file in os.scandir(algo_dir):
        env_dir = env_file.path
        env_name = env_dir.split("/")[-1]
        model_dict[algo][env_name] = {}
        env_dict[algo][env_name] = {}
        env, original_model = load_zoo_agent(env_name, algo, zoo_path = path_to_zoo)
        env_dict[algo][env_name] = env
        model_dict[algo][env_name]['original'] = original_model
        for pkl_file in os.scandir(env_dir):
            post_name = pkl_file.path.split("/")[-1].split(".")[0]
            model_dict[algo][env_name][post_name] = ALGOS[algo].load(pkl_file.path, env=env, device='cpu')

/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/td3/HalfCheetahBulletEnv-v0_1/HalfCheetahBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/td3/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0


/home/ty/miniconda3/envs/policy_refinement/lib/python3.6/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/td3/HopperBulletEnv-v0_1/HopperBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/td3/AntBulletEnv-v0_1/AntBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/td3/ReacherBulletEnv-v0_1/ReacherBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/tqc/HalfCheetahBulletEnv-v0_1/HalfCheetahBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/tqc/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/tqc/HopperBulletEnv-v0_1/HopperBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/tqc/AntBulletEnv-v0_1/AntBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/tqc/ReacherBulletEnv-v0_1/ReacherBulletEnv-v0
/home/ty/Documents/GitHub/rl-baselines3-zoo//rl-trained-agents/a2c/HalfCheetahBulletEnv-v0_1/HalfCheetahBulletEnv-v0
Loading running avera

In [5]:
algo_list_og = list(model_dict.keys())
env_list = list(model_dict[algo_list[0]].keys())
post_list = list(model_dict[algo_list[0]][env_list[0]].keys())

print(algo_list_og) # Original Algorithm
print(env_list) # Environment

# Postprocessor, autogenerated names. "original" means the original agent from zoo without additional training, "postprocess_default" means just ARS with not extra reward.
# Any other name is some new reward functio 
print(post_list)
algo_list=['a2c', 'ppo', 'ddpg', 'td3', 'sac', 'tqc']
print(algo_list)

['td3', 'tqc', 'a2c', 'sac', 'ddpg', 'ppo']
['HalfCheetahBulletEnv-v0', 'Walker2DBulletEnv-v0', 'HopperBulletEnv-v0', 'AntBulletEnv-v0', 'ReacherBulletEnv-v0']
['original', 'postprocess_default']
['a2c', 'ppo', 'ddpg', 'td3', 'sac', 'tqc']


In [6]:
#from seagul.zoo3_utils import do_rollout_stable
def do_rollout_mine(env, model, render=False, render_wait=0.01, seed=None):
    state_list = []
    act_list = []
    reward_list = []

    if seed:
        env.seed(seed)
    obs = env.reset()
    done = False

    while not done:
        # Add push or something else here 

        state_list.append(np.copy(obs))
        

        actions,_ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(actions)
        
        # Or maybe here

    
        if render:
            env.render()
            time.sleep(render_wait)
        
        act_list.append(np.copy(actions))
        reward_list.append(reward)

    state_arr = np.stack(state_list).squeeze()
    act_arr = np.stack(act_list).squeeze()
    
    return state_arr, act_arr, reward_list, info


In [ ]:
# Rollout for all with no noise for 100 trials

data_all_no_noise={}
row = 0
n_trial = 100 #Trial number
for k in enumerate(post_list):
    for j in enumerate(env_list):
        for i in enumerate(algo_list):
            env = env_dict[i[1]][j[1]]
            model = model_dict[i[1]][j[1]][k[1]] # no refinement
            #model2 = model_dict[i[1]][j[1]][post_list[1]] # ARS policy refinement
            count = 1
            fail = 0
            rewards_list = []
            l_list = [] 
            
            while count <= n_trial :
                obs,act,rew,info = do_rollout_mine(env, model)
                ep_rew = info[0]['episode']['r']
                ep_l = info[0]['episode']['l']

                rewards_list.append(ep_rew)
                l_list.append(ep_l)

                rewards_arr = np.stack(rewards_list).squeeze()
                l_arr = np.stack(l_list).squeeze()

                if info[0]['episode']['l']<1000:
                    fail+=1
                count+=1
            fail_percentage = 100*fail/n_trial

            reward_mean = np.mean(rewards_arr)
            reward_std = np.std(rewards_arr)
            l_mean = np.mean(l_arr)
    #        if k[0] == 1:
    #            data_no_noise[str(row)] = data_no_noise[str(row)]+[reward_mean,reward_std,fail_percentage,l_mean]
    #        else:
    #            data_no_noise[str(row)] = [env_id, post_name[k[0]],i[1],reward_mean,reward_std,fail_percentage,l_mean]

            data_all_no_noise[str(row)] = [k[1],j[1],i[1],reward_mean,reward_std,fail_percentage,l_mean]
            print(row)
            row += 1
df_all_no_noise = pd.DataFrame.from_dict(data_all_no_noise, orient='index', columns=['post', 'config.env_name', 'config.algo', 'ureward_mean', 'ureward_std', 'fail/100ep', 'len_mean'])
#df_all_no_noise = pd.DataFrame.from_dict(data_all_no_noise, orient='index', columns=['env', 'post', 'algo', 'r_mean', 'r_std', 'fall/100ep before', 'l_mean', 'r_mean_refined', 'r_std_refined', 'fall/100ep after', 'l_mean_refined'])

#df_all_no_noise['algo'] = df_all_no_noise['algo'].str.upper()
path = '~/Documents/GitHub/policy_refinement/Ty_files/Ty_csv/all_no_noise.csv'
df_all_no_noise.to_csv(path,index = False)
df_all_no_noise

0
1
2


In [13]:
df_all_no_noise

,post,config.env_name,config.algo,ureward_mean,ureward_std,fail/100ep,len_mean
0,original,HalfCheetahBulletEnv-v0,a2c,2093.934339,0.0,0.0,1000.0
1,original,HalfCheetahBulletEnv-v0,ppo,2902.401609,0.0,0.0,1000.0
2,original,HalfCheetahBulletEnv-v0,ddpg,2107.880258,0.0,0.0,1000.0
3,original,HalfCheetahBulletEnv-v0,td3,2813.343944,0.0,0.0,1000.0
4,original,HalfCheetahBulletEnv-v0,sac,2770.250149,0.0,0.0,1000.0
5,original,HalfCheetahBulletEnv-v0,tqc,3679.345899,0.0,0.0,1000.0
6,original,Walker2DBulletEnv-v0,a2c,995.005505,0.0,0.0,1000.0
7,original,Walker2DBulletEnv-v0,ppo,2099.273594,0.0,0.0,1000.0
8,original,Walker2DBulletEnv-v0,ddpg,43.352394,0.0,100.0,59.0
9,original,Walker2DBulletEnv-v0,td3,2253.697485,0.0,0.0,1000.0
